#Data Cleaning

Data cleaning was done on individual statement dataframes which were then merged together for further cleaning. 

#Load libraries and data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [ ]:
cur_path = "/content/drive/MyDrive/Colab Notebooks/Capstone/"

## Balance Sheets

In [ ]:
bs_df = pd.read_csv(cur_path+'bs_df3.csv')
bs_df2 = pd.read_csv(cur_path+'bs_df4.csv')
bal_sheet = pd.concat([bs_df, bs_df2])
bal_sheet.drop('Unnamed: 0', axis=1, inplace=True)
bal_sheet.dropna(inplace=True)
bal_sheet.reset_index(inplace=True)
bal_sheet.drop('index', axis=1, inplace=True)
bal_sheet.head()

,cik,dates,scale,total assets,total liabilities,total shareholders equity
0,814926,"Sep. 30, 2012",ones,"7,625,322","6,301,100","1,324,222"
1,814926,"Dec. 31, 2011",ones,"4,138,431","2,499,758","1,638,673"
2,818677,"Sep. 30, 2012",ones,"904,235,504","821,098,610","83,136,894"
3,818677,"Mar. 31, 2012",ones,"924,636,608","843,864,722","80,771,886"
4,820608,"Sep. 30, 2012",ones,"610,116","786,884","(176,768)"


## Income Statements

In [ ]:
is_df = pd.read_csv(cur_path+'is_df3.csv')
is_df2 = pd.read_csv(cur_path+'is_df4.csv')
inc_state = pd.concat([is_df, is_df2])
inc_state.drop('Unnamed: 0', axis=1, inplace=True)
inc_state.dropna(inplace=True)
inc_state.reset_index(inplace=True)
inc_state.drop('index', axis=1, inplace=True)
inc_state.head()

,cik,scale,dates,net income
0,818677,ones,"Sep. 30, 2012","$ 669,790"
1,818677,ones,"Sep. 30, 2011","$ 418,453"
2,828678,thousands,"Sep. 30, 2012",(33)
3,828678,thousands,"Sep. 30, 2011",(524)
4,837852,ones,"Sep. 30, 2012","$ (4,437,427)"


## Statements of Cash Flows

In [ ]:
cf_df = pd.read_csv(cur_path+'cf_df3.csv')
cf_df2 = pd.read_csv(cur_path+'cf_df4.csv')
cash_flow = pd.concat([cf_df, cf_df2])
cash_flow.drop('Unnamed: 0', axis=1, inplace=True)
cash_flow.dropna(inplace=True)
cash_flow.reset_index(inplace=True)
cash_flow.drop('index', axis=1, inplace=True)
cash_flow.head()

,cik,scale,dates,cash equivalents
0,814586,ones,"Sep. 30, 2012","$ 2,379,565"
1,814586,ones,"Sep. 30, 2011","$ 860,683"
2,814926,ones,"Sep. 30, 2012","77,692"
3,814926,ones,"Sep. 30, 2011","66,464"
4,818677,ones,"Sep. 30, 2012","14,408,177"


#Balance Sheet Cleaning

In [ ]:
#find the indices of rows that did not have real dates and remove them using the index_lst
index_lst=[]
for i in range(len(bal_sheet)):
  date =str(bal_sheet['dates'].iloc[i])
  #dates in this current format are 13 characters long
  if (len(date)!=13):
    index_lst.append(i)
#drop rows
bal_sheet.drop(index_lst, inplace=True)
bal_sheet.reset_index(inplace=True)
bal_sheet.head()

,index,cik,dates,scale,total assets,total liabilities,total shareholders equity
0,0,814926,"Sep. 30, 2012",ones,"7,625,322","6,301,100","1,324,222"
1,1,814926,"Dec. 31, 2011",ones,"4,138,431","2,499,758","1,638,673"
2,2,818677,"Sep. 30, 2012",ones,"904,235,504","821,098,610","83,136,894"
3,3,818677,"Mar. 31, 2012",ones,"924,636,608","843,864,722","80,771,886"
4,4,820608,"Sep. 30, 2012",ones,"610,116","786,884","(176,768)"


In [ ]:
#May didn't have a period after the name of the month which messed up the datetime conversion
#I add a period after the name of the month to correct for that
for i in range(len(bal_sheet)):
  date = str(bal_sheet['dates'].iloc[i])
  day = date[5:7]
  year = date[9:13]
  if 'May' in date:
    bal_sheet['dates'].iloc[i] = 'May. '+day+", "+year

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
#convert dates column from string to datetime
bal_sheet['dates'] = pd.to_datetime(bal_sheet['dates'], format='%b. %d, %Y') 

In [ ]:
#add quarters for each row based on month
qtr_lst=[]
for i in range(len(bal_sheet)):
  month = bal_sheet['dates'].iloc[i].month
  if (month >1) and (month <=4):
    qtr_lst.append(1)
  elif (month >4) and (month <=7):
    qtr_lst.append(2)
  elif (month >7) and (month <=10):
    qtr_lst.append(3)
  else:
        qtr_lst.append(4)
bal_sheet.insert(3, 'qtr', qtr_lst)

In [ ]:
#remove all rows with odd index which were the previous period values from each balance sheet
index_lst=[]
for i in range(len(bal_sheet)):
  if i%2!=0:
    index_lst.append(i)
bal_sheet.drop(index_lst, inplace=True)

In [ ]:
#add a years column
years=[]
for i in range(len(bal_sheet)):
  years.append(bal_sheet['dates'].iloc[i].year)
bal_sheet.insert(3, 'year', years)

In [ ]:
#order by cik, year, and qtr
bal_sheet.sort_values(by=['cik', 'year', 'qtr'], inplace=True)

In [ ]:
#reset index and remove unneccessary columns
bal_sheet.reset_index(inplace=True)
bal_sheet.drop(['level_0', 'index'], axis=1, inplace=True)
bal_sheet.head()

,cik,dates,year,qtr,scale,total assets,total liabilities,total shareholders equity
0,1750,2012-11-30,2012,4,millions,"2,189.3","$ 2,189.3",902.8
1,1750,2013-08-31,2013,3,millions,"2,129.2","$ 2,129.2",938.3
2,1750,2014-08-31,2014,3,millions,"2,205.4","$ 2,205.4","1,003.8"
3,1750,2014-11-30,2014,4,millions,"2,227.7us-gaap_Assets","$ 2,227.7us-gaap_LiabilitiesAndStockholdersEquity","1,001.3us-gaap_StockholdersEquity"
4,1961,2013-12-31,2013,4,ones,"328,044","$ 328,044","(4,001,698)"


#Income Statement Cleaning

The steps for income statement cleaning and cash flows cleaning is identical to the balance sheet cleaning except for a few things which I will mark. 

In [ ]:
index_lst=[]
for i in range(len(inc_state)):
  date =str(inc_state['dates'].iloc[i])
  if (len(date)>13) | (len(date)<10):
    index_lst.append(i)
inc_state.drop(index_lst, inplace=True)
inc_state.reset_index(inplace=True)
inc_state.head()

,index,cik,scale,dates,net income
0,0,818677,ones,"Sep. 30, 2012","$ 669,790"
1,1,818677,ones,"Sep. 30, 2011","$ 418,453"
2,2,828678,thousands,"Sep. 30, 2012",(33)
3,3,828678,thousands,"Sep. 30, 2011",(524)
4,4,837852,ones,"Sep. 30, 2012","$ (4,437,427)"


In [ ]:
#I was getting errors when cleaning the May dates so I had to add in an extra condition
#This was to account for single digit days
for i in range(len(inc_state)):
  date = str(inc_state['dates'].iloc[i])
  #Single digit days had fewer characters
  if len(date)<13:
    day = date[4:6]
    year = date[8:12]
  else:
    day = date[5:7]
    year = date[9:13]
  if 'May' in date:
    inc_state['dates'].iloc[i] = 'May. '+day+", "+year

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
inc_state['dates'] = pd.to_datetime(inc_state['dates'], format='%b. %d, %Y') 

In [ ]:
qtr_lst=[]
for i in range(len(inc_state)):
  month = inc_state['dates'].iloc[i].month
  if (month >1) and (month <=4):
    qtr_lst.append(1)
  elif (month >4) and (month <=7):
    qtr_lst.append(2)
  elif (month >7) and (month <=10):
    qtr_lst.append(3)
  else:
        qtr_lst.append(4)
inc_state.insert(3, 'qtr', qtr_lst)

In [ ]:
index_lst=[]
for i in range(len(inc_state)):
  if i%2!=0:
    index_lst.append(i)
inc_state.drop(index_lst, inplace=True)

In [ ]:
years=[]
for i in range(len(inc_state)):
  years.append(inc_state['dates'].iloc[i].year)
inc_state.insert(3, 'year', years)

In [ ]:
inc_state.sort_values(by=['cik', 'year', 'qtr'], inplace=True)

In [ ]:
inc_state.reset_index(inplace=True)
inc_state.drop(['level_0', 'index'], axis=1, inplace=True)
inc_state.head()

,cik,scale,year,qtr,dates,net income
0,1750,millions,2012,4,2012-11-30,$ 17.9
1,1750,millions,2013,1,2013-02-28,$ 18.5
2,1750,millions,2013,4,2013-11-30,$ 20.0
3,1750,millions,2014,1,2014-02-28,$ 18.0
4,1750,millions,2014,1,2014-02-28,$ 18.0us-gaap_ProfitLoss


#Cash Flows Cleaning
The only difference in this cleaning was with the May dates. This issue was the same as the income statements. 

In [ ]:
index_lst=[]
for i in range(len(cash_flow)):
  date =str(cash_flow['dates'].iloc[i])
  if (len(date)>13) | (len(date)<10):
    index_lst.append(i)
cash_flow.drop(index_lst, inplace=True)
cash_flow.reset_index(inplace=True)

In [ ]:
for i in range(len(cash_flow)):
  date = str(cash_flow['dates'].iloc[i])
  if len(date)<13:
    day = date[4:6]
    year = date[8:12]
  else:
    day = date[5:7]
    year = date[9:13]
  if 'May' in date:
    cash_flow['dates'].iloc[i] = 'May. '+day+", "+year

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
cash_flow['dates'] = pd.to_datetime(cash_flow['dates'], format='%b. %d, %Y') 

In [ ]:
qtr_lst=[]
for i in range(len(cash_flow)):
  month = cash_flow['dates'].iloc[i].month
  if (month >1) and (month <=4):
    qtr_lst.append(1)
  elif (month >4) and (month <=7):
    qtr_lst.append(2)
  elif (month >7) and (month <=10):
    qtr_lst.append(3)
  else:
        qtr_lst.append(4)
cash_flow.insert(3, 'qtr', qtr_lst)

In [ ]:
index_lst=[]
for i in range(len(cash_flow)):
  if i%2!=0:
    index_lst.append(i)
cash_flow.drop(index_lst, inplace=True)

In [ ]:
years=[]
for i in range(len(cash_flow)):
  years.append(cash_flow['dates'].iloc[i].year)
cash_flow.insert(3, 'year', years)

In [ ]:
cash_flow.sort_values(by=['cik', 'year', 'qtr'], inplace=True)

In [ ]:
cash_flow.reset_index(inplace=True)
cash_flow.drop(['level_0', 'index'], axis=1, inplace=True)
cash_flow.head()

,cik,scale,year,qtr,dates,cash equivalents
0,1750,millions,2012,4,2012-11-30,$ 69.2
1,1750,millions,2012,4,2012-11-30,$ 69.2
2,1750,millions,2013,1,2013-02-28,$ 47.4
3,1750,millions,2013,3,2013-08-31,$ 90.9
4,1750,millions,2013,3,2013-08-31,$ 90.9


#Merging dataframes
I found the doing an outer join was the best way to keep all columns. I joined each dataframe on cik, year, and quarter since I was working with quarterly documents. 

In [ ]:
#merge balance sheets with income statements
merged = bal_sheet.merge(inc_state, how='outer', left_on=['cik', 'qtr', 'year'], right_on=['cik', 'qtr', 'year']).dropna()
merged.rename(columns={'dates_x': 'dates', 'scale_x':'scale'},inplace=True)
merged.reset_index(inplace=True)
#drop duplicate/unneccessary columns
merged.drop(['dates_y', 'scale_y', 'index'], axis=1, inplace=True)
merged.head()

,cik,dates,year,qtr,scale,total assets,total liabilities,total shareholders equity,net income
0,1750,2012-11-30,2012,4,millions,"2,189.3","$ 2,189.3",902.8,$ 17.9
1,1750,2014-11-30,2014,4,millions,"2,227.7us-gaap_Assets","$ 2,227.7us-gaap_LiabilitiesAndStockholdersEquity","1,001.3us-gaap_StockholdersEquity",$ 15.2us-gaap_ProfitLoss
2,2034,2015-12-31,2015,4,thousands,"524,394","$ 238,537","285,857","$ 8,270"
3,2034,2016-03-31,2016,1,thousands,"544,600","$ 245,749","298,851","$ 10,424"
4,2098,2013-09-30,2013,3,thousands,"70,738","36,286","34,452",$ 959


In [ ]:
#merge cash flows with the other two
merged = cash_flow.merge(merged, how='outer', left_on=['cik', 'qtr', 'year'], right_on=['cik', 'qtr', 'year']).dropna()
merged.rename(columns={'dates_x': 'dates', 'scale_x':'scale'}, inplace=True)
merged.reset_index(inplace=True)
merged.drop(['dates_y', 'scale_y', 'index'], axis=1, inplace=True)
merged.head()

,cik,scale,year,qtr,dates,cash equivalents,total assets,total liabilities,total shareholders equity,net income
0,1750,millions,2012,4,2012-11-30,$ 69.2,"2,189.3","$ 2,189.3",902.8,$ 17.9
1,1750,millions,2012,4,2012-11-30,$ 69.2,"2,189.3","$ 2,189.3",902.8,$ 17.9
2,2034,thousands,2015,4,2015-12-31,"$ 54,349","524,394","$ 238,537","285,857","$ 8,270"
3,2034,thousands,2016,1,2016-03-31,"$ 53,093","544,600","$ 245,749","298,851","$ 10,424"
4,2098,thousands,2013,3,2013-09-30,"$ 11,809","70,738","36,286","34,452",$ 959


In [ ]:
#Removing all non-numeric characters so I can convert values to numeric later
index_lst=[]
for i in range(5,10,1):
  for j in range(len(merged)):
    merged.iloc[j,i]=merged.iloc[j,i].replace("$ ","").replace("(", "-").replace(")","").replace(",", "")
    #some rows had weird values and I assumed they were all longer than 20 characters
    if len(merged.iloc[j,i]) > 20:
      index_lst.append(j)
merged.drop(index_lst, inplace=True)
merged.head()

,cik,scale,year,qtr,dates,cash equivalents,total assets,total liabilities,total shareholders equity,net income
0,1750,millions,2012,4,2012-11-30,69.2,2189.3,2189.3,902.8,17.9
1,2034,thousands,2015,4,2015-12-31,54349,524394,238537,285857,8270
2,2034,thousands,2016,1,2016-03-31,53093,544600,245749,298851,10424
3,2098,thousands,2013,3,2013-09-30,11809,70738,36286,34452,959
4,2098,thousands,2015,3,2015-09-30,4621,86585,42878,43707,1208


In [ ]:
#reset the index
merged.reset_index(inplace=True)
merged.drop('index', axis=1, inplace=True)

In [ ]:
#change columns to numeric values
for i in range(5,10,1):
  merged[merged.columns[i]] = merged[merged.columns[i]].astype(float)
merged.head()

,cik,scale,year,qtr,dates,cash equivalents,total assets,total liabilities,total shareholders equity,net income
0,1750,millions,2012,4,2012-11-30,69.2,2189.3,2189.3,902.8,17.9
1,2034,thousands,2015,4,2015-12-31,54349.0,524394.0,238537.0,285857.0,8270.0
2,2034,thousands,2016,1,2016-03-31,53093.0,544600.0,245749.0,298851.0,10424.0
3,2098,thousands,2013,3,2013-09-30,11809.0,70738.0,36286.0,34452.0,959.0
4,2098,thousands,2015,3,2015-09-30,4621.0,86585.0,42878.0,43707.0,1208.0


In [ ]:
#multiply by the scale the company reports in so all values are in ones
for i in range(len(merged)):
  if merged['scale'].iloc[i]=='thousands':
    merged.iloc[i, 5:10]=merged.iloc[i, 5:10]*1000
  elif merged['scale'].iloc[i]=='millions':
    merged.iloc[i, 5:10]=merged.iloc[i, 5:10]*1000000
merged.head()

,cik,scale,year,qtr,dates,cash equivalents,total assets,total liabilities,total shareholders equity,net income
0,1750,millions,2012,4,2012-11-30,69200000.0,2.189300e+09,2.189300e+09,902800000.0,17900000.0
1,2034,thousands,2015,4,2015-12-31,54349000.0,5.243940e+08,2.385370e+08,285857000.0,8270000.0
2,2034,thousands,2016,1,2016-03-31,53093000.0,5.446000e+08,2.457490e+08,298851000.0,10424000.0
3,2098,thousands,2013,3,2013-09-30,11809000.0,7.073800e+07,3.628600e+07,34452000.0,959000.0
4,2098,thousands,2015,3,2015-09-30,4621000.0,8.658500e+07,4.287800e+07,43707000.0,1208000.0


In [ ]:
#check cik, year, qtr (i.e. document) duplicates 
df = merged.groupby(['cik','year', 'qtr']).size().reset_index().rename(columns={0:'count'})
duplicates = df[df['count']>1]

In [ ]:
#remove document duplicates by matching them with the cik, year, and quarter from the duplicates dataframe
index_lst=[]
for i in range(len(duplicates)):
  cik = duplicates['cik'].iloc[i]
  year = duplicates['year'].iloc[i]
  qtr=duplicates['qtr'].iloc[i]
  count = duplicates['count'].iloc[i]
  for j in range(len(merged)):
    if (merged['cik'].iloc[j] == cik) and (merged['year'].iloc[j] == year) and (merged['qtr'].iloc[j] == qtr):
      index_range= list(range(j+1, j+count, 1))
      for item in index_range:
        index_lst.append(item)
      break

merged.drop(index_lst, inplace=True)
merged.reset_index(inplace=True)
merged.drop('index', axis=1, inplace=True)

In [ ]:
merged.to_csv(cur_path+'reports2.csv', index=False)